# 知识处理Pipeline - 专家级实现

**方案**: 向量库+JSON双存储  
**技术栈**: Qwen3-Embedding + deepseek-v3 + Chroma  
**质量标准**: 遵循[AI行为约束规范](../../docs/AI行为约束规范.md)

## 流程

```
PDF/Word/PPT文件（51个）
 ↓ [模块1] KnowledgeOrganizer
17个主题组
 ↓ [模块2] DocumentLoader  
List[Document]
 ↓ [模块3] KnowledgeExtractor (deepseek-v3)
结构化JSON
 ↓ [模块4] VectorStoreManager (Qwen3-Embedding)
Chroma向量库
 ↓ [模块5] KnowledgeProcessor
完整知识库
```


In [1]:
# ========== 环境准备 ==========

# 标准库
import re
import json
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass
from difflib import SequenceMatcher
from datetime import datetime

# LangChain - Document Loaders
from langchain_community.document_loaders import (
    PyMuPDFLoader,           # PDF加载（推荐）
    Docx2txtLoader,          # Word加载  
    UnstructuredPowerPointLoader  # PPT加载
)

# LangChain - Core
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

# LangChain - Embeddings & VectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# LangChain - LLM（技术决策：deepseek-v3）
from langchain_deepseek import ChatDeepSeek

# 进度显示
from tqdm.auto import tqdm

# 日志
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ 所有依赖导入完成")

# 环境变量
from dotenv import load_dotenv
load_dotenv("../../../config/.env")

print("✅ 依赖导入完成")


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


: 

## 配置参数


In [ ]:
# ========== 全局配置 ==========

# 路径配置
KNOWLEDGE_BASE_DIR = Path("../knowledge_base")
OUTPUT_DIR = Path("./output")
VECTOR_DB_DIR = OUTPUT_DIR / "vector_db"
STRUCTURED_JSON_DIR = OUTPUT_DIR / "structured_knowledge"

# 模型配置（按技术决策）
# Embedding模型：使用本地路径（避免网络下载）
EMBEDDING_MODEL_PATH = "/Users/Qunying/.cache/huggingface/hub/models--Qwen--Qwen3-Embedding-0.6B/snapshots"
# 获取最新的snapshot目录
import os
snapshot_dirs = os.listdir(EMBEDDING_MODEL_PATH)
EMBEDDING_MODEL = os.path.join(EMBEDDING_MODEL_PATH, snapshot_dirs[0]) if snapshot_dirs else "Qwen/Qwen3-Embedding-0.6B"

LLM_MODEL = "deepseek-reasoner"  # 决策#005
LLM_TEMPERATURE = 0  # 确保输出稳定性

# 文本分割配置
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

# 创建输出目录
OUTPUT_DIR.mkdir(exist_ok=True)
VECTOR_DB_DIR.mkdir(exist_ok=True)
STRUCTURED_JSON_DIR.mkdir(exist_ok=True)

logger.info(f"📁 知识库路径: {KNOWLEDGE_BASE_DIR}")
logger.info(f"📁 输出路径: {OUTPUT_DIR}")
logger.info(f"🤖 Embedding: {EMBEDDING_MODEL}")
logger.info(f"🤖 LLM: {LLM_MODEL}")

print("✅ 配置完成")


2025-12-03 15:33:29,936 - INFO - 📁 知识库路径: ../knowledge_base
2025-12-03 15:33:29,936 - INFO - 📁 输出路径: output
2025-12-03 15:33:29,936 - INFO - 🤖 Embedding: /Users/Qunying/.cache/huggingface/hub/models--Qwen--Qwen3-Embedding-0.6B/snapshots/c54f2e6e80b2d7b7de06f51cec4959f6b3e03418
2025-12-03 15:33:29,937 - INFO - 🤖 LLM: deepseek-reasoner


✅ 配置完成


## 数据结构定义


In [ ]:
# ========== 数据结构定义（符合专家标准）==========

from enum import IntEnum

class FilePriority(IntEnum):
    """文件优先级枚举
    
    优先级规则：
    - PDF笔记最优先（最详细）
    - Word文档次之
    - 普通PDF第三
    - PPT最后（信息密度低）
    """
    PDF_NOTE = 1      # PDF笔记文件（文件名包含"笔记"）
    WORD_DOC = 2      # Word文档
    PDF_REGULAR = 3   # 普通PDF文件
    POWERPOINT = 4    # PowerPoint文件
    UNKNOWN = 99      # 未知类型


@dataclass
class FileInfo:
    """文件信息数据类
    
    Attributes:
        path: 文件完整路径
        original_name: 原始文件名
        cleaned_name: 清洗后的文件名（去除噪音）
        sequence: 序号（整数）
        sequence_str: 序号字符串
        priority: 文件优先级
    """
    path: Path
    original_name: str
    cleaned_name: str
    sequence: int
    sequence_str: str
    priority: FilePriority


@dataclass
class KnowledgeGroup:
    """知识组数据类
    
    Attributes:
        group_key: 组唯一标识
        topic: 主题名称
        sequence: 主题序号
        files: 组内所有文件
        primary_file: 主要文件（优先级最高）
        file_types: 文件类型列表
    """
    group_key: str
    topic: str
    sequence: int
    files: List[FileInfo]
    primary_file: FileInfo
    file_types: List[str]


print("✅ 数据结构定义完成")


✅ 数据结构定义完成


## 步骤1: KnowledgeOrganizer - 文件扫描分组


In [ ]:
class KnowledgeOrganizer:
    """知识文件扫描和智能分组
    
    功能:
    1. 扫描知识库目录,支持PDF/Word/PPT
    2. 清洗文件名(去除时间戳、噪音标记等)
    3. 按相似度智能分组(同主题的不同文件类型)
    4. 按优先级排序(PDF笔记 > Word > PDF > PPT)
    
    分组算法:
    - 提取序号(如"01第一节")作为主键
    - 计算文件名相似度
    - 相似度超过阈值的归为一组
    """
    
    # 支持的文件扩展名
    SUPPORTED_EXTENSIONS = {'.pdf', '.doc', '.docx', '.ppt', '.pptx'}
    
    # 文件名噪音模式(正则表达式)
    NOISE_PATTERNS = [
        r'\[防断更.*?\]',  # [防断更微coc36666]
        r'\[.*?微.*?\]',   # [微信号xxx]
        r'_\d{14}',        # _20250706193405
        r'_\d{8}',         # _20250706
        r'_笔记',          # _笔记
        r'\s*\(.*?\)\s*'   # (备注)
    ]

    def __init__(self, 
                 knowledge_base_dir: str, 
                 similarity_threshold: float = 0.7, 
                 verbose: bool = True):
        """初始化知识组织器
        
        Args:
            knowledge_base_dir: 知识库根目录
            similarity_threshold: 文件名相似度阈值(0-1),默认0.7
            verbose: 是否打印详细日志
        
        Raises:
            ValueError: 目录不存在时抛出
        """
        self.knowledge_base_dir = Path(knowledge_base_dir)
        self.similarity_threshold = similarity_threshold
        self.verbose = verbose
        if not self.knowledge_base_dir.exists():
            raise ValueError(f"目录不存在: {self.knowledge_base_dir}")

    def _log(self, msg):
        if self.verbose: print(msg)

    def clean_filename(self, filename):
        """清洗文件名(去除噪音)"""
        name = Path(filename).stem
        for pattern in self.NOISE_PATTERNS:
            name = re.sub(pattern, '', name)
        return re.sub(r'\s+', ' ', name).strip()

    def extract_sequence_number(self, filename):
        """提取序号"""
        match = re.match(r'^(\d+)', filename)
        return (int(match.group(1)), match.group(1)) if match else (999999, "")

    def calculate_similarity(self, str1, str2):
        """计算相似度"""
        return SequenceMatcher(None, str1, str2).ratio()

    def get_file_priority(self, file_path):
        """确定文件优先级"""
        name, suffix = file_path.name.lower(), file_path.suffix.lower()
        if '笔记' in name and suffix == '.pdf': return FilePriority.PDF_NOTE
        if suffix in ['.doc', '.docx']: return FilePriority.WORD_DOC
        if suffix == '.pdf': return FilePriority.PDF_REGULAR
        if suffix in ['.ppt', '.pptx']: return FilePriority.POWERPOINT
        return FilePriority.UNKNOWN

    def create_file_info(self, file_path):
        """创建文件信息对象"""
        original_name = file_path.name
        cleaned_name = self.clean_filename(original_name)
        sequence, sequence_str = self.extract_sequence_number(cleaned_name)
        priority = self.get_file_priority(file_path)
        return FileInfo(file_path, original_name, cleaned_name, sequence, sequence_str, priority)

    def group_files_by_similarity(self, files):
        """按相似度分组"""
        groups, processed = {}, set()
        for i, file1 in enumerate(files):
            if file1.path in processed: continue
            group_key = f"{file1.sequence_str}_{file1.cleaned_name[:20]}"
            group_files = [file1]
            processed.add(file1.path)

            for file2 in files[i+1:]:
                if file2.path in processed: continue
                if file1.sequence == file2.sequence:
                    if self.calculate_similarity(file1.cleaned_name, file2.cleaned_name) >= self.similarity_threshold:
                        group_files.append(file2)
                        processed.add(file2.path)

            group_files.sort(key=lambda f: (f.priority.value, f.original_name))
            groups[group_key] = KnowledgeGroup(group_key, file1.cleaned_name, file1.sequence,
                                              group_files, group_files[0], [f.path.suffix for f in group_files])
            self._log(f"✓ {file1.cleaned_name[:30]} ({len(group_files)}文件)")
        return groups

    def scan_and_organize(self):
        """扫描并组织文件"""
        self._log(f"📚 扫描: {self.knowledge_base_dir}")
        all_files = []
        for ext in self.SUPPORTED_EXTENSIONS:
            all_files.extend(self.knowledge_base_dir.glob(f"*{ext}"))

        if not all_files:
            self._log("⚠️ 未找到文件")
            return {}

        self._log(f"找到 {len(all_files)} 个文件")
        file_infos = [self.create_file_info(f) for f in all_files]
        groups = self.group_files_by_similarity(file_infos)
        sorted_groups = dict(sorted(groups.items(), key=lambda x: x[1].sequence))
        self._log(f"✅ {len(sorted_groups)} 个知识块\n")
        return {self.knowledge_base_dir.name: sorted_groups}

print("✅ 步骤1完成")


✅ 步骤1完成


In [ ]:
class DocumentLoader:
    """文档加载器
    
    支持多种文档格式加载:
    - PDF: PyMuPDFLoader (推荐,性能最佳)
    - Word: Docx2txtLoader (.doc, .docx)
    - PowerPoint: UnstructuredPowerPointLoader (.ppt, .pptx)
    """

    def load_pdf(self, file_path: Path) -> List[Document]:
        """加载PDF文件
        
        Args:
            file_path: PDF文件路径
            
        Returns:
            文档列表(每页一个Document)
        """
        try:
            loader = PyMuPDFLoader(str(file_path))
            return loader.load()
        except Exception as e:
            logger.error(f"PDF加载失败 {file_path.name}: {e}")
            return []

    def load_word(self, file_path: Path) -> List[Document]:
        """加载Word文件
        
        Args:
            file_path: Word文件路径
            
        Returns:
            文档列表
        """
        try:
            loader = Docx2txtLoader(str(file_path))
            return loader.load()
        except Exception as e:
            logger.error(f"Word加载失败 {file_path.name}: {e}")
            return []

    def load_ppt(self, file_path: Path) -> List[Document]:
        """加载PowerPoint文件
        
        Args:
            file_path: PPT文件路径
            
        Returns:
            文档列表
        """
        try:
            loader = UnstructuredPowerPointLoader(str(file_path))
            return loader.load()
        except Exception as e:
            logger.error(f"PPT加载失败 {file_path.name}: {e}")
            return []

    def clean_document_text(self, doc: Document) -> Document:
        """清洗文档文本
        
        清理内容:
        - 特殊字符
        - 多余空白
        - 噪音内容
        
        Args:
            doc: 原始文档
            
        Returns:
            清洗后的文档
        """
        text = doc.page_content
        text = re.sub(r'[\uf06c\uf0fc]', '', text)  # 特殊字符
        text = re.sub(r'\s+', ' ', text)  # 多余空白
        doc.page_content = text.strip()
        return doc

    def load_and_clean(self, file_path: Path) -> List[Document]:
        """加载并清洗文档(统一入口)
        
        Args:
            file_path: 文件路径
            
        Returns:
            清洗后的文档列表
        """
        suffix = file_path.suffix.lower()
        
        # 根据文件类型选择加载器
        if suffix == '.pdf':
            docs = self.load_pdf(file_path)
        elif suffix in ['.doc', '.docx']:
            docs = self.load_word(file_path)
        elif suffix in ['.ppt', '.pptx']:
            docs = self.load_ppt(file_path)
        else:
            logger.warning(f"不支持的文件类型: {suffix}")
            return []
        
        # 清洗文档
        if docs:
            docs = [self.clean_document_text(doc) for doc in docs]
            logger.info(f"✓ 加载 {file_path.name}: {len(docs)}页")
        
        return docs

print("✅ 步骤2完成")


✅ 步骤2完成


## 步骤3: KnowledgeExtractor - LLM结构化提取


In [ ]:
class KnowledgeExtractor:
    """LLM知识提取器
    
    使用LLM从文档中提取结构化知识(JSON格式)
    技术决策: 使用deepseek-v3模型进行知识提取
    """

    def __init__(self, model_name: str = LLM_MODEL, temperature: float = LLM_TEMPERATURE):
        """初始化知识提取器
        
        Args:
            model_name: LLM模型名称,默认使用全局配置
            temperature: 温度参数,默认0确保输出稳定性
        """
        self.llm = ChatDeepSeek(model=model_name, temperature=temperature)
        logger.info(f"🤖 LLM初始化: {model_name}")
        self.prompt = ChatPromptTemplate.from_template("""
你是金融知识提取专家。从文档提取结构化知识,返回JSON。

文档: {content}

提取JSON(只返回JSON):
{{
  "topic": "主题",
  "key_concepts": [{{"name": "概念", "definition": "定义", "importance": "重要性"}}],
  "indicators": [{{"name": "指标", "calculation": "计算", "interpretation": "解读"}}],
  "analysis_methods": [{{"name": "方法", "steps": "步骤", "application": "应用"}}],
  "summary": "总结"
}}
""")

    def extract_from_documents(self, docs: List[Document], topic: str) -> Dict:
        """从文档提取知识"""
        content = "\n\n".join([d.page_content for d in docs[:5]])[:15000]
        try:
            chain = self.prompt | self.llm | JsonOutputParser()
            result = chain.invoke({"content": content})
            result["topic"] = topic
            return result
        except Exception as e:
            print(f"⚠️ LLM提取失败: {e}")
            return {"topic": topic, "key_concepts": [], "indicators": [],
                   "analysis_methods": [], "summary": "提取失败"}

print("✅ 步骤3完成")


✅ 步骤3完成


## 步骤4: VectorStoreManager - 向量化存储


In [ ]:
class VectorStoreManager:
    """向量存储管理器
    
    负责文档向量化和Chroma向量数据库管理
    技术决策: 使用Qwen3-Embedding-0.6B模型进行向量化
    """

    def __init__(self, 
                 embedding_model: str = EMBEDDING_MODEL, 
                 persist_directory: str = str(VECTOR_DB_DIR)):
        """初始化向量存储管理器
        
        Args:
            embedding_model: Embedding模型名称,默认使用全局配置
            persist_directory: 向量数据库持久化目录
        """
        self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
        self.persist_directory = Path(persist_directory)
        logger.info(f"🤖 Embedding初始化: {embedding_model}")
        self.vector_stores = {}

    def get_or_create_store(self, domain: str) -> Chroma:
        """获取或创建向量存储"""
        if domain in self.vector_stores:
            return self.vector_stores[domain]

        persist_path = str(self.persist_directory / domain)
        store = Chroma(
            collection_name=f"{domain}_col",
            embedding_function=self.embeddings,
            persist_directory=persist_path
        )
        self.vector_stores[domain] = store
        return store

    def split_documents(self, docs: List[Document]) -> List[Document]:
        """分割文档为chunks"""
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200, add_start_index=True
        )
        return splitter.split_documents(docs)

    def add_documents(self, domain: str, docs: List[Document], metadata: Dict = None):
        """添加文档到向量存储"""
        if not docs:
            print("  ⚠️ 无文档,跳过向量化")
            return

        chunks = self.split_documents(docs)
        if metadata:
            for chunk in chunks:
                chunk.metadata.update(metadata)

        store = self.get_or_create_store(domain)
        store.add_documents(chunks)
        print(f"  → 向量化: {len(chunks)} chunks")

print("✅ 步骤4完成")


✅ 步骤4完成


## 步骤5: KnowledgeProcessor - 完整Pipeline


In [ ]:
class KnowledgeProcessor:
    """知识处理Pipeline协调器
    
    整合5个核心模块,执行完整的知识处理流程:
    1. 文件扫描分组 (KnowledgeOrganizer)
    2. 文档加载 (DocumentLoader)
    3. 知识提取 (KnowledgeExtractor)
    4. 向量化存储 (VectorStoreManager)
    5. JSON存储
    """

    def __init__(self, 
                 knowledge_base_dir: str, 
                 memories_dir: str = str(STRUCTURED_JSON_DIR), 
                 vector_db_dir: str = str(VECTOR_DB_DIR)):
        """初始化Pipeline协调器
        
        Args:
            knowledge_base_dir: 知识库根目录
            memories_dir: JSON结构化知识存储目录
            vector_db_dir: 向量数据库存储目录
        """
        self.organizer = KnowledgeOrganizer(knowledge_base_dir)
        self.loader = DocumentLoader()
        self.extractor = KnowledgeExtractor()
        self.vector_manager = VectorStoreManager(persist_directory=vector_db_dir)
        self.memories_dir = Path(memories_dir)
        logger.info("🚀 Pipeline协调器初始化完成")

    def save_to_memories(self, domain: str, group_key: str, knowledge: Dict):
        """保存知识到JSON"""
        domain_dir = self.memories_dir / domain
        domain_dir.mkdir(parents=True, exist_ok=True)
        json_file = domain_dir / f"{group_key}.json"
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(knowledge, f, ensure_ascii=False, indent=2)
        print(f"  → JSON: {json_file.name}")

    def process_all(self, limit: int = None):
        """处理所有知识文件"""
        print("\n" + "="*80)
        print("🚀 开始完整Pipeline")
        print("="*80 + "\n")

        organized = self.organizer.scan_and_organize()

        for domain, groups in organized.items():
            print(f"\n📂 领域: {domain}")
            print("-" * 80)

            count = 0
            for group_key, group in groups.items():
                if limit and count >= limit:
                    print(f"\n⏸️  达到限制({limit}),停止")
                    break

                print(f"\n[{count+1}] {group.topic}")

                # 加载
                docs = self.loader.load_and_clean(group.primary_file.path)
                if not docs:
                    print("  ⚠️ 加载失败")
                    continue
                print(f"  → 加载: {len(docs)} 页")

                # 提取
                knowledge = self.extractor.extract_from_documents(docs, group.topic)
                self.save_to_memories(domain, group_key, knowledge)

                # 向量化
                self.vector_manager.add_documents(domain, docs,
                    {"domain": domain, "topic": group.topic, "seq": group.sequence})

                count += 1

            print(f"\n✅ {domain}: 完成 {count} 个")

        print("\n" + "="*80)
        print("🎉 Pipeline完成!")
        print("="*80)
        print(f"\n📁 输出:")
        print(f"  - JSON: {self.memories_dir}")
        print(f"  - 向量库: {self.vector_manager.persist_directory}")

print("✅ 步骤5完成")


✅ 步骤5完成


In [ ]:
# ========== Pipeline测试 ==========

# 初始化Processor(使用全局配置)
processor = KnowledgeProcessor(
    knowledge_base_dir=str(KNOWLEDGE_BASE_DIR)
)

print("✅ Processor初始化完成")
print(f"📁 知识库: {KNOWLEDGE_BASE_DIR}")
print(f"📁 JSON输出: {STRUCTURED_JSON_DIR}")
print(f"📁 向量库: {VECTOR_DB_DIR}")


2025-12-03 15:33:30,131 - INFO - 🤖 LLM初始化: deepseek-reasoner
2025-12-03 15:33:30,134 - INFO - Use pytorch device_name: mps
2025-12-03 15:33:30,134 - INFO - Load pretrained SentenceTransformer: /Users/Qunying/.cache/huggingface/hub/models--Qwen--Qwen3-Embedding-0.6B/snapshots/c54f2e6e80b2d7b7de06f51cec4959f6b3e03418
2025-12-03 15:33:34,327 - INFO - 1 prompt is loaded, with the key: query
2025-12-03 15:33:34,328 - INFO - 🤖 Embedding初始化: /Users/Qunying/.cache/huggingface/hub/models--Qwen--Qwen3-Embedding-0.6B/snapshots/c54f2e6e80b2d7b7de06f51cec4959f6b3e03418
2025-12-03 15:33:34,329 - INFO - 🚀 Pipeline协调器初始化完成


✅ Processor初始化完成
📁 知识库: ../knowledge_base
📁 JSON输出: output/structured_knowledge
📁 向量库: output/vector_db


In [ ]:
# 测试: 处理前2个知识块
processor.process_all(limit=2)


2025-12-03 15:33:34,396 - INFO - ✓ 加载 01第一节 中国经济的“三驾马车”[防断更微coc36666]_笔记.pdf: 4页



🚀 开始完整Pipeline

📚 扫描: ../knowledge_base
找到 51 个文件
✓ 14.第十四节 汇率投资手册 (3文件)
✓ 07第七节 物价——快速入门读懂经 (3文件)
✓ 11.第十一节 基金投资手册 (3文件)
✓ 12.第十二节 保险投资手册 (3文件)
✓ 08第八节 如何读懂经济周期 (3文件)
✓ 15.第十五节 大宗商品投资手册 (3文件)
✓ 05第五节 PMI——快速入门读懂经济形势 (3文件)
✓ 06第六节 金融——快速入门读懂经济形势 (3文件)
✓ 17.第十七节 格雷厄姆：华尔街教父 (3文件)
✓ 13.第十三节 黄金投资手册 (3文件)
✓ 03第三节 投资——快速入门读懂经济形势 (3文件)
✓ 04第四节 出口——快速入门读懂经济形势 (3文件)
✓ 09第九节 看懂投资时钟，踩准投资节奏 (3文件)
✓ 16.第十六节 房地产投资手册 (3文件)
✓ 01第一节 中国经济的“三驾马车” (3文件)
✓ 02第二节 消费——快速入门读懂经济形势 (3文件)
✓ 10第十节 股市投资手册 (3文件)
✅ 17 个知识块


📂 领域: knowledge_base
--------------------------------------------------------------------------------

[1] 01第一节 中国经济的“三驾马车”
  → 加载: 4 页


2025-12-03 15:33:34,616 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"


In [ ]:
# 正式运行: 处理所有知识块(取消注释)
# processor.process_all()


## 检查输出


In [ ]:
# 查看生成的JSON
memories_path = Path("./memories/knowledge")
if memories_path.exists():
    for domain_dir in memories_path.iterdir():
        if domain_dir.is_dir():
            print(f"\n📂 {domain_dir.name}:")
            for json_file in sorted(domain_dir.glob("*.json")):
                print(f"  - {json_file.name}")
else:
    print("⚠️ 尚未生成输出")


In [ ]:
# 读取第一个JSON查看
json_files = list(Path("./memories/knowledge").rglob("*.json"))
if json_files:
    with open(json_files[0], 'r', encoding='utf-8') as f:
        sample = json.load(f)
    print(f"示例 ({json_files[0].name}):")
    print(json.dumps(sample, ensure_ascii=False, indent=2))
else:
    print("⚠️ 无JSON")


## 总结

### ✅ 功能完整

- 步骤1: 文件扫描分组 ✓
- 步骤2: 文档加载清洗 ✓ (仅PDF)
- 步骤3: LLM结构化提取 ✓
- 步骤4: 向量化存储 ✓
- 步骤5: Pipeline协调 ✓

### 📦 输出

- `memories/knowledge/{domain}/*.json` - 结构化知识
- `vector_db/{domain}/` - 向量数据库

### 🚀 使用

1. 先用`limit=2`测试
2. 确认后`process_all()`处理全部
3. 检查输出结果

### ⚙️ 说明

- **仅支持PDF**: 简化版只处理PDF(主要格式)
- **需API Key**: DeepSeek API (在.env配置)
- **首次下载**: embedding模型会自动下载
